# 2-Dataset

到这里我们便完成了对于 MiniMind Tokenizer 和 Model 部分的全部了解，我们所熟悉的大语言模型正是由这个组件构成的，接下来，我们需要对大模型训练所使用的数据集结构有个基本的认识。

想要训练一个能够正常对话，并且符合人类对话偏好的大模型一般需要经过以下几个训练阶段：

- 预训练（Pre-training）
- 有监督微调（Supervised Fine-tuning，SFT）
- 人类反馈强化学习（Reinforcement Learning from Human Feedback，RLHF）

在不同训练阶段使用的数据集有所不同，下面会从 MiniMind 代码出发进行介绍和解读。

In [1]:
import json
import random
import re

import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import train_test_split
import os
import ast
from transformers import AutoTokenizer

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 为了避免警告信息

In [2]:
# 从 ../model 目录加载分词器
tokenizer = AutoTokenizer.from_pretrained('./model/minimind_tokenizer')
print(tokenizer.vocab_size)

6400


## I.预训练数据集

预训练是模型在大规模语料上进行无监督学习的训练阶段，在该阶段，模型主要学习下一词预测的能力，简单的来说就是学会说话，而不是胡言乱语。因此，该阶段训练的模型不会具有问答能力，而是根据用户输入进行简单的词语接龙。

我们可以看一看预训练的数据集格式：

```
{"text": "如何才能摆脱拖延症？ 治愈拖延症并不容易，但以下建议可能有所帮助..."}
```

为了降低该 demo 的运行门槛，在 `./demo` 文件夹下提供了包含两条训练数据的 `pretrain_data.jsonl` 文件作为熟悉训练流程的数据集 demo。

In [3]:
# 我们可以查看一下 demo 中提供的数据
path_pretrain = './toydata/pretrain_data.jsonl'

with open(path_pretrain, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        data = json.loads(line.strip())
        print(f'Row {line_num}: {data}\n')

Row 1: {'text': 'LLM首先要学习的并非直接与人交流，而是让网络参数中充满知识的墨水，“墨水” 理论上喝的越饱越好，产生大量的对世界的知识积累。 预训练就是让Model先埋头苦学大量基本的知识，例如从Wiki百科、新闻、书籍整理大规模的高质量训练数据。 这个过程是“无监督”的，即人类不需要在过程中做任何“有监督”的校正，而是由模型自己从大量文本中总结规律学习知识点。 模型此阶段目的只有一个：学会词语接龙。例如我们输入“秦始皇”四个字，它可以接龙“是中国的第一位皇帝”。'}

Row 2: {'text': '经过预训练，LLM此时已经掌握了大量知识，然而此时它只会无脑地词语接龙，还不会与人聊天。 SFT阶段就需要把半成品LLM施加一个自定义的聊天模板进行微调。 例如模型遇到这样的模板【问题->回答，问题->回答】后不再无脑接龙，而是意识到这是一段完整的对话结束。 称这个过程为指令微调，就如同让已经学富五车的「牛顿」先生适应21世纪智能手机的聊天习惯，学习屏幕左侧是对方消息，右侧是本人消息这个规律。 在训练时，MiniMind的指令和回答长度被截断在512，是为了节省显存空间。就像我们学习时，会先从短的文章开始，当学会写作200字作文后，800字文章也可以手到擒来。 在需要长度拓展时，只需要准备少量的2k/4k/8k长度对话数据进行进一步微调即可（此时最好配合RoPE-NTK的基准差值）。'}



我们知道，构建一个深度学习数据集需要继承 `torch.utils.data.dataset`，并构建 DataLoader 数据迭代器进行迭代访问。下面，我们来看看 MiniMind 是如何抽象一个预训练数据集的。

In [4]:
class PretrainDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_length=512):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = self.load_data(data_path)

    # 按行读取 jsonl 文件，并存储在列表中
    def load_data(self, path):
        samples = []
        with open(path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                data = json.loads(line.strip())  # 逐行读取并解析 JSON，line.strip() 去除首尾空白字符
                samples.append(data)
        return samples

    # 获取数据集大小
    def __len__(self):
        return len(self.samples)

    # 获取指定索引的数据样本
    def __getitem__(self, index):
        sample = self.samples[index]

        text = f"{self.tokenizer.bos_token}{str(sample['text'])}{self.tokenizer.eos_token}"
        print(f'样本 {index} 的文本内容: {text}')  # 打印文本内容以供调试
        # 对文本进行编码，得到 input_ids
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',  # 填充到最大长度
            truncation=True,  # 截断超过 max_length 的部分
            return_tensors='pt'  # 返回 PyTorch 张量
        )
        # 提取 input_ids 并去掉多余的维度，即 batch 维度
        input_ids = encoding.input_ids.squeeze()  # shape: (max_length,)
        loss_mask = (input_ids != self.tokenizer.pad_token_id) # 生成 loss mask，非填充部分为 True
        # 拆分为输入input序列X和目标target序列Y
        X = input_ids[:-1] # <input> 不包含最后一个 token EOS
        Y = input_ids[1:] # <target> 不包含第一个 token BOS
        loss_mask = loss_mask[1:] # loss mask 对齐 <target> 序列，非填充部分为 1
        return X, Y, loss_mask

In [5]:
pretrain_dataset = PretrainDataset(path_pretrain, tokenizer)  # 实例化数据集
print(f'预训练数据集长度{len(pretrain_dataset)}')
x, y, lm = pretrain_dataset[0]
print(x.shape, y.shape, lm.shape)

预训练数据集长度2
样本 0 的文本内容: <s>LLM首先要学习的并非直接与人交流，而是让网络参数中充满知识的墨水，“墨水” 理论上喝的越饱越好，产生大量的对世界的知识积累。 预训练就是让Model先埋头苦学大量基本的知识，例如从Wiki百科、新闻、书籍整理大规模的高质量训练数据。 这个过程是“无监督”的，即人类不需要在过程中做任何“有监督”的校正，而是由模型自己从大量文本中总结规律学习知识点。 模型此阶段目的只有一个：学会词语接龙。例如我们输入“秦始皇”四个字，它可以接龙“是中国的第一位皇帝”。</s>
torch.Size([511]) torch.Size([511]) torch.Size([511])


## II.有监督微调数据集

有监督微调（Supervised Fine Tuning，SFT）对预训练后得到的基座 LLM 施加一个自定义聊天模板进行微调，由于在这一阶段，模型训练的目标是根据用户指令生成响应（构建问答体系），故又称为指令微调。

我们可以看一看有监督微调的数据集格式：

```
{
    "conversations": [
        {"role": "user", "content": "你好"},
        {"role": "assistant", "content": "你好！"},
        {"role": "user", "content": "再见"},
        {"role": "assistant", "content": "再见！"}
    ]
}
```

为了降低该 demo 的运行门槛，在 `./demo` 文件夹下提供了包含两条 conversation 问答数据的 `sft_data.jsonl` 文件作为熟悉训练流程的数据集 demo。

In [6]:
# 我们可以查看一下 demo 中提供的数据
path_sft = './toydata/sft_data.jsonl'

with open(path_sft, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        data = json.loads(line.strip())
        print(f'Row {line_num}: {data}\n')

Row 1: {'conversations': [{'role': 'user', 'content': '你好吗？'}, {'role': 'assistant', 'content': '我很好，谢谢！你呢？'}, {'role': 'user', 'content': '我也很好，谢谢！'}, {'role': 'assistant', 'content': '太好了！祝你今天愉快！'}]}

Row 2: {'conversations': [{'role': 'user', 'content': '你喜欢什么运动？'}, {'role': 'assistant', 'content': '我喜欢跑步和游泳。你呢？'}, {'role': 'user', 'content': '我喜欢打篮球！'}, {'role': 'assistant', 'content': '篮球很棒！是一个很好的团队运动。'}]}



接下来，我们尝试构造一个数据集对象，实现对 sft 格式数据的读取与处理。

In [7]:
class SFTDataset(Dataset):
    def __init__(self, jsonl_path, tokenizer, max_length=512):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = self.load_data(jsonl_path)
        # 定义助手assistant回答的开始和结束标记的 ID 列表，存在长度不为1的可能性，因此要用len()处理长度
        self.bos_id = tokenizer(f'{tokenizer.bos_token}assistant\n', add_special_tokens=False).input_ids # 设定 bos token
        self.eos_id = tokenizer(f'{tokenizer.eos_token}\n', add_special_tokens=False).input_ids # 设定 eos token

    # 按行读取 jsonl 文件，每个样本（字典）中必须包含一个 conversations 键，即对话列表
    def load_data(self, path):
        samples = []
        with open(path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                data = json.loads(line.strip())
                samples.append(data)
        return samples
        
    # 获取数据集大小    
    def __len__(self):
        return len(self.samples)

    # 构建符合 ChatML 格式的对话
    def _create_chat_prompt(self, conversations):
        messages = []
        for i, turn in enumerate(conversations):
            # 交替分配角色：偶数轮为 user，奇数轮为 assistant
            role = 'user' if i % 2 == 0 else 'assistant'
            messages.append({"role": role, "content": turn['content']})
        # 应用分词器的聊天模板，生成最终的提示文本
        # 例如，初始对话是 [{"role": "user", "content": "你好"}, {"role": "assistant", "content": "你好！有什么可以帮你？"}]
        # 最终生成的 prompt 可能是：<s>user\n你好</s><s>assistant\n你好！有什么可以帮你？</s>
        return self.tokenizer.apply_chat_template(
            messages,  
            tokenize=False,  # 不进行分词，返回字符串
            add_generation_prompt=False  # 不添加生成提示符（如 "<|assistant|>"）
        )

    # 生成 loss mask，只对助手回答部分计算损失
    def _generate_loss_mask(self, input_ids):
        loss_mask = [0] * len(input_ids)  # 初始化全为 0 的 loss mask
        i = 0
        while i < len(input_ids):
            if input_ids[i:i + len(self.bos_id)] == self.bos_id: # 检查是否到达助手回答的开始标记
                # 找到对应的 eos token 位置
                start = i + len(self.bos_id) # 助手回答开始位置
                end = start  # 初始化助手回答结束位置
                # 向后遍历，找到对应的助手回答结束标记（eos_id）
                while end < len(input_ids):
                    if input_ids[end:end + len(self.eos_id)] == self.eos_id: 
                        break
                    end += 1
                # 将助手回答部分的 loss mask 设为 1
                for j in range(start + 1, min(end + len(self.eos_id) + 1, self.max_length)): # 忽略超过最大输入长度的 tokens
                    loss_mask[j] = 1  # 助手回答的有效内容，设为1（计算损失），包含了 eos token
                i = end + len(self.eos_id) if end < len(input_ids) else len(input_ids) # 更新 i 以退出当前对话轮次
            else:
                i += 1
        return loss_mask
    
    # 获取指定索引的数据样本
    def __getitem__(self, index):
        sample = self.samples[index]
        prompt = self._create_chat_prompt(sample['conversations']) # 构建 ChatML 格式的对话提示
        # 示例：原始对话是 ["你好", "你好！有什么可以帮你？"]，转换后是 <s>user\n你好</s><s>assistant\n你好！有什么可以帮你？</s>
        print(prompt)  # 输出构建的对话提示
        input_ids = self.tokenizer(prompt).input_ids[:self.max_length] # 编码并截断到最大长度
        input_ids += [self.tokenizer.pad_token_id] * (self.max_length - len(input_ids)) # 填充：不足最大长度的部分用pad token补齐
        # print(input_ids) # 输出 input_ids 列表
        
        loss_mask = self._generate_loss_mask(input_ids)  # 生成 loss mask，只对助手回答部分计算损失

        X = torch.tensor(input_ids[:-1], dtype=torch.long)
        Y = torch.tensor(input_ids[1:], dtype=torch.long)
        loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long) # loss mask 对齐 <target> 序列，非填充部分为 1

        return X, Y, loss_mask

In [8]:
sft_dataset = SFTDataset(path_sft, tokenizer)  # 实例化数据集
print(f'SFT 数据集长度 = {len(sft_dataset)}')
x, y, lm = sft_dataset[0]
print(f'样本 shape = {x.shape}, 标签 shape = {y.shape}, loss_mask shape {lm.shape}')
# 打印 loss mask，你会发现在序列中有两处以 1 填充的序列，这是因为我们在一个 conversation 中开展了两轮对话，其中只有 assistant 回复计算损失
print(lm) 

SFT 数据集长度 = 2
<s>system
你是 MiniMind，是一个有用的人工智能助手。</s>
<s>user
你好吗？</s>
<s>assistant
我很好，谢谢！你呢？</s>
<s>user
我也很好，谢谢！</s>
<s>assistant
太好了！祝你今天愉快！</s>

样本 shape = torch.Size([511]), 标签 shape = torch.Size([511]), loss_mask shape torch.Size([511])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## III.人类反馈强化学习数据集

在 MiniMind 项目中，采用直接偏好优化（Direct Parameter Optimization，DPO）训练大模型对齐人类偏好。在这一训练阶段，模型将会根据提供的问答正反例进行偏好优化，从而降低让人类不满意的答案出现的几率。

与PPO(Proximal Policy Optimization)这种需要奖励模型、价值模型的RL算法不同； DPO通过推导PPO奖励模型的显式解，把在线奖励模型换成离线数据，Ref模型输出可以提前保存。 DPO性能几乎不变，只用跑 actor_model 和 ref_model 两个模型，大大节省显存开销和增加训练稳定性。

---

PPO（近端策略优化）是经典强化学习算法，靠奖励模型与Actor - Critic框架驱动策略更新，稳定可控但流程复杂；DPO（直接偏好优化）是2023年提出的大模型对齐方法，跳过奖励模型，直接用偏好数据优化，流程简化、训练高效稳定。下面从核心原理、训练流程、关键特性、适用场景等方面详细介绍并对比。

#### PPO（Proximal Policy Optimization，近端策略优化）
PPO是OpenAI于2017年提出的强化学习算法，核心是在策略梯度框架下，通过限制策略更新幅度保证训练稳定，避免单次更新过大导致模型崩溃，广泛用于大模型RLHF流程。
1. **核心原理**
    - **Actor - Critic架构**：包含策略网络（Actor）生成内容，价值网络（Critic）评估状态价值，结合奖励模型计算优势函数（如GAE）来优化策略。
    - **裁剪机制**：将新旧策略的概率比裁剪在[1 - ε, 1 + ε]（常见ε=0.2），限制更新幅度。
    - **KL散度惩罚**：约束新策略与参考模型（如SFT模型）的偏离，防止生成质量下降。
2.  **训练流程**
    1.  策略模型生成样本（Rollout）。
    2.  奖励模型（RM）对样本打分。
    3.  价值网络计算优势函数。
    4.  用裁剪目标与KL惩罚更新策略与价值网络，反复迭代。
3.  **优缺点**
    |优点|缺点|
    | ---- | ---- |
    |适配复杂奖励信号，干预能力强，适合安全与风格剧变需求|依赖奖励模型，训练流程复杂，需协调Actor、Critic、RM等，计算成本高，调参复杂|
    |训练稳定性较好，通过裁剪与KL惩罚减少崩溃风险|易出现奖励黑客、模式崩溃等问题，需精细调参|

#### DPO（Direct Preference Optimization，直接偏好优化）
DPO是2023年斯坦福提出的大模型对齐方法，跳过奖励模型，直接用人类偏好的对比数据优化策略，把偏好比较转化为概率优化问题，大幅简化流程。
1.  **核心原理**
    - **偏好直接建模**：基于Bradley - Terry模型，将“$y_w$优于$y_l$”转化为概率问题，优化目标是让模型对$y_w$的概率高于$y_l$。
    - **损失函数**：最大化偏好响应相对非偏好响应的概率，同时用KL散度约束与参考模型（如SFT模型）的偏离，公式为$L_{DPO}(\pi;\pi_{ref}) = -\mathbb{E}_{(x,y_w,y_l)\sim D}\left[\log\sigma\left(\frac{1}{\beta}\left(\log\frac{\pi(y_w|x)}{\pi_{ref}(y_w|x)}-\log\frac{\pi(y_l|x)}{\pi_{ref}(y_l|x)}\right)\right)\right]$，β控制偏离程度。
2.  **训练流程**
    1.  准备偏好数据（输入提示$x$ + 优质回答$y_w$ + 劣质回答$y_l$）。
    2.  计算当前模型与参考模型对$y_w$、$y_l$的概率比。
    3.  用DPO损失函数更新策略模型，无需额外价值网络与奖励模型。
3.  **优缺点**
    |优点|缺点|
    | ---- | ---- |
    |无需奖励模型，流程简单，训练高效稳定，显存开销低|对参考模型质量敏感，偏好数据需高质量对比对，难以融合多源复杂奖励信号|
    |调参简单，超参数少（如β），复现性好，适合后期精调|干预强度相对弱，难应对剧烈风格转变等强干预场景|


#### PPO与DPO核心区别对比表
|对比维度|PPO|DPO|
| ---- | ---- | ---- |
|核心依赖|必须有奖励模型（RM）与价值网络（Critic）|无需奖励模型与Critic，直接用偏好对比数据|
|优化方式|奖励驱动的强化学习，“拉扯式”强干预|偏好直接优化，“排序式”温和精调|
|训练流程|Rollout→RM打分→计算优势→更新Actor/Critic，多模型协同|偏好数据输入→对比概率→更新策略，单模型优化|
|计算成本|高（多模型训练与迭代）|低（仅优化策略模型）|
|适用场景|安全收紧、风格剧变、复杂奖励信号任务|后期精调、效率优先、偏好对齐场景，如对话系统|

---

我们可以看一看人类反馈强化学习的数据集格式：

```
{
  "chosen": [
    {"content": "Query", "role": "user"}, 
    {"content": "good answer", "role": "assistant"}
  ], 
  "rejected": [
    {"content": "Query", "role": "user"}, 
    {"content": "bad answer", "role": "assistant"}
  ]
}
```

为了降低该 demo 的运行门槛，在 ./demo 文件夹下提供了包含两条 conversation 问答数据的 dpo_data.jsonl 文件作为熟悉训练流程的数据集 demo。

In [9]:
# 我们可以查看一下 demo 中提供的数据
path_dpo = './toydata/dpo_data.jsonl'

with open(path_dpo, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        data = json.loads(line.strip())
        print(f'Row {line_num}: {data}\n')

Row 1: {'chosen': [{'content': '你好吗？', 'role': 'user'}, {'content': '我很好，谢谢！你呢？', 'role': 'assistant'}, {'content': '今天过得怎么样？', 'role': 'user'}, {'content': '挺好的，去跑步了，心情不错。', 'role': 'assistant'}], 'rejected': [{'content': '你好吗？', 'role': 'user'}, {'content': '不好，我很累。', 'role': 'assistant'}, {'content': '你喜欢什么运动？', 'role': 'user'}, {'content': '我不喜欢运动，没兴趣。', 'role': 'assistant'}]}

Row 2: {'chosen': [{'content': '你喜欢旅行吗？', 'role': 'user'}, {'content': '我喜欢，尤其是去海边。', 'role': 'assistant'}, {'content': '你去过哪些地方？', 'role': 'user'}, {'content': '我去过欧洲、美国和日本，都是很棒的经历。', 'role': 'assistant'}], 'rejected': [{'content': '你喜欢旅行吗？', 'role': 'user'}, {'content': '不太喜欢，觉得太麻烦。', 'role': 'assistant'}, {'content': '你去过哪些地方？', 'role': 'user'}, {'content': '没去过太多地方，主要是待在家里。', 'role': 'assistant'}]}



接下来，我们尝试构造 json 对象，实现对 dpo 格式数据的读取和处理

In [10]:
class DPODataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=512):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.padding = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0  # 设置填充标记 ID
        # 定义模型回答的前缀和后缀token ID（用于识别模型生成的内容）
        self.bos_id = tokenizer(f'{tokenizer.bos_token}assistant\n', add_special_tokens=False).input_ids # 内容前缀
        self.eos_id = tokenizer(f'{tokenizer.eos_token}\n', add_special_tokens=False).input_ids # 内容后缀
        with open(file_path, 'r', encoding='utf-8') as f: # 加载数据
            self.data = []
            for line in f:
                line = line.strip()
                obj = json.loads(line)
                self.data.append(obj)

    # 获取数据集大小
    def __len__(self):
        return len(self.data)

    # 生成 loss mask，只对助手回答部分计算损失
    def _generate_loss_mask(self, input_ids):
        """此处的损失掩码生成函数与 SFT 阶段逻辑一致"""
        loss_mask = [0] * len(input_ids)
        i = 0
        while i < len(input_ids):
            if input_ids[i:i + len(self.bos_id)] == self.bos_id:
                start = i + len(self.bos_id)
                end = start
                while end < len(input_ids):
                    if input_ids[end:end + len(self.eos_id)] == self.eos_id:
                        break
                    end += 1
                for j in range(start + 1, min(end + len(self.eos_id) + 1, self.max_length)):
                    loss_mask[j] = 1
                i = end + len(self.eos_id) if end < len(input_ids) else len(input_ids)
            else:
                i += 1
        return loss_mask
    
    # 获取指定索引的数据样本
    def __getitem__(self, index):
        item = self.data[index]
        chosen = item['chosen'] # 优选回答：list of {role, content}（如[{"role": "user", ...}, {"role": "assistant", ...}]）
        rejected = item['rejected'] # 拒绝回答，格式同上

        # 用tokenizer的对话模板格式化（转为连贯文本），格式同sft阶段
        chosen_prompt = self.tokenizer.apply_chat_template(
            chosen, tokenize=False, add_generation_prompt=False
        )
        print(chosen_prompt) # 输出格式化后的提示文本
        rejected_prompt = self.tokenizer.apply_chat_template(
            rejected, tokenize=False, add_generation_prompt=False
        )
        print(rejected_prompt) # 输出格式化后的提示文本
        # 对格式化后的文本进行分词，转为token ID，并统一长度
        chosen_encoding = self.tokenizer(
            chosen_prompt, truncation=True, max_length=self.max_length, padding='max_length'
        )
        rejected_encoding = self.tokenizer(
            rejected_prompt, truncation=True, max_length=self.max_length, padding='max_length'
        )
        # 拆分为输入和目标序列，并生成对应的 loss mask
        chosen_input_ids = chosen_encoding['input_ids']  # 提取chosen的 token ID 列表
        chosen_loss_mask = self._generate_loss_mask(chosen_input_ids)  # 生成对应的 loss mask
        rejected_input_ids = rejected_encoding['input_ids']  # 提取rejected的 token ID 列表
        rejected_loss_mask = self._generate_loss_mask(rejected_input_ids)  # 生成对应的 loss mask
        # 同sft阶段的输入输出构建方式
        x_chosen = torch.tensor(chosen_input_ids[:-1], dtype=torch.long)
        y_chosen = torch.tensor(chosen_input_ids[1:], dtype=torch.long)
        mask_chosen = torch.tensor(chosen_loss_mask[1:], dtype=torch.long)
        x_rejected = torch.tensor(rejected_input_ids[:-1], dtype=torch.long)
        y_rejected = torch.tensor(rejected_input_ids[1:], dtype=torch.long)
        mask_rejected = torch.tensor(rejected_loss_mask[1:], dtype=torch.long)

        return {
            'x_chosen': x_chosen,
            'y_chosen': y_chosen,
            'mask_chosen': mask_chosen,
            'x_rejected': x_rejected,
            'y_rejected': y_rejected,
            'mask_rejected': mask_rejected
        }

In [11]:
dpo_dataset = DPODataset(path_dpo, tokenizer)
print(f'DPO 数据集长度：{len(dpo_dataset)}')
res = dpo_dataset[0]
print(res) # 输出res中的元素

DPO 数据集长度：2
<s>system
你是 MiniMind，是一个有用的人工智能助手。</s>
<s>user
你好吗？</s>
<s>assistant
我很好，谢谢！你呢？</s>
<s>user
今天过得怎么样？</s>
<s>assistant
挺好的，去跑步了，心情不错。</s>

<s>system
你是 MiniMind，是一个有用的人工智能助手。</s>
<s>user
你好吗？</s>
<s>assistant
不好，我很累。</s>
<s>user
你喜欢什么运动？</s>
<s>assistant
我不喜欢运动，没兴趣。</s>

{'x_chosen': tensor([   1,   85,  736,  201,  608,  345,  562,  261,   75,   47,  807,  270,
        1589,  400,  411, 1946,  740, 1728,  945, 1184,  286,    2,  201,    1,
         320,  275,  201, 5134, 2235,  814,    2,  201,    1, 1078,  538,  501,
         201,  397, 5675,  270, 4191, 2207,  608, 3274,  814,    2,  201,    1,
         320,  275,  201, 4668,  580,  776, 5027, 1355,  814,    2,  201,    1,
        1078,  538,  501,  201,  550,  121, 1188,  270, 1473, 4286, 1242,  446,
         270,  854,  811, 5409,  286,    2,  201,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,  

## IV.AI反馈的强化学习数据集

- 使用**AI模型**（通常是预训练的语言奖励模型）来提供反馈，而不直接依赖人类的人工标注。
- 这里的“AI”也可以是某些规则奖励，例如数学答案/代码解释器...


| 类型    | 裁判 | 优点        | 缺点         |
|-------|----|-----------|------------|
| RLHF  | 人类 | 更贴近真实人类偏好 | 成本高、效率低    |
| RLAIF | 模型 | 自动化、可扩展性强 | 可能偏离人类真实偏好 |

二者本质上是一样的，都是通过**强化学习的方式**，利用某种形式的"**反馈**"来优化模型的行为。

除了**反馈**的来源不同，其他并无任何区别。

相比RLHF依赖人类标注chosen/rejected偏好对，RLAIF则完全由AI来充当"裁判"。  
所谓AI"裁判"可以是model-base的奖励大模型(Reward Model)，也可以是R1一样设置规则函数进行校验，也可以是例如工具调用的环境反馈。  
例如：数学题答案是否正确、工具调用执行代码能否通过测试用例、推理过程是否符合格式...都可以自动化判断。  
RLAIF的最大优势在于**可扩展性**和**On-Policy**的特点——不需要昂贵的人工标注，可以生成海量的训练样本，让模型在在线大量试错中快速进化。

---

GRPO（Group Relative Policy Optimization，群体相对策略优化）是由DeepSeek团队于2024年提出的强化学习算法，核心是通过组内相对奖励信号替代传统PPO的价值函数（Critic），以提升LLM偏好对齐与推理能力微调的稳定性和效率，已在DeepSeek - R1等模型训练中取得显著成效。以下从核心原理、算法流程、损失函数、关键特性、与相关算法对比及应用场景展开详细介绍。

### 一、核心思想与动机
- **核心思想**：对于同一prompt生成的一组（G个）回复，利用组内相对排序/奖励差异作为训练信号，而非依赖绝对奖励分数或单独训练的价值函数，契合人类“相对比较更可靠”的判断本质。
- **核心动机**
    - 规避PPO中奖励模型（RM）噪声、尺度漂移导致的训练不稳定与reward hacking问题。
    - 移除Critic网络，降低计算开销，适配LLM大规模训练资源约束。
    - 利用多回复组内信息，提升偏好数据利用率，尤其适配多候选排序的偏好标注场景。

### 二、算法核心流程
1. **组采样（Group Sampling）**：对每个prompt，从当前策略π_old采样G个回复{o₁, o₂, …, o_G}，组成一个样本组。
2.  **奖励评估**：通过人工标注、自动评估工具（如数学推理结果验证器）或RM，获取组内各回复的奖励{r₁, r₂, …, r_G}，支持全序、部分序或二元判断。
3.  **组内归一化与优势估计**：计算组内奖励均值μ_r与标准差σ_r，对每个回复的奖励做标准化得到优势A_i = (r_i - μ_r)/σ_r；无σ_r时可仅用均值做中心化，以此替代Critic的价值估计。
4.  **策略更新**：采用类似PPO的裁剪目标，结合KL正则防止偏离参考策略π_ref（如SFT模型），更新策略π_new以提高高优势回复的生成概率。
5.  **迭代优化**：重复采样 - 评估 - 归一化 - 更新流程，直至策略收敛或性能达标。

### 三、核心损失函数
GRPO的目标函数融合相对偏好优化与策略约束，常见形式如下：
$$
L_{GRPO} = \mathbb{E}_{q \sim \mu, \{o_i\} \sim \pi_{old}(\cdot|q)} \left[ \sum_{i=1}^G \min\left( \frac{\pi_{new}(o_i|q)}{\pi_{old}(o_i|q)} A_i, \text{clip}\left( \frac{\pi_{new}(o_i|q)}{\pi_{old}(o_i|q)}, 1 - \epsilon, 1 + \epsilon \right) A_i \right) - \beta \cdot KL(\pi_{new}(\cdot|q) \parallel \pi_{ref}(\cdot|q)) \right]
$$
- 其中，ε为裁剪系数（通常取0.2），β为KL正则系数，用于平衡策略更新幅度与参考分布一致性。
- 当组内仅含2个回复（G=2）时，GRPO损失可退化为类似DPO的成对对比损失，体现其对DPO的扩展能力。

### 四、关键特性
|特性|说明|
| ---- | ---- |
|**无需Critic**|用组内归一化替代价值函数，减少计算与内存开销，降低超参数调优复杂度|
|**训练稳定性高**|相对奖励信号梯度方差低，不受绝对奖励尺度漂移影响，抗噪声能力强|
|**偏好信息高效利用**|支持多回复排序（G>2），比二元对（DPO/PPO常用）蕴含更丰富偏好信息|
|**适配多类型奖励**|兼容二元、部分序、全序及可验证奖励（如推理任务的对错标记）|
|**与参数高效微调兼容**|可与LoRA等方法结合，仅更新部分参数即可实现高效对齐|

### 五、与PPO、DPO的核心差异
|对比维度|GRPO|PPO|DPO|
| ---- | ---- | ---- | ---- |
|奖励依赖|组内相对奖励，无需绝对分数|依赖RM输出的绝对奖励|二元成对偏好（chosen/rejected）|
|是否需Critic|无需，组内归一化替代|必需，用于价值估计|无需，直接优化成对对比损失|
|偏好数据形式|支持G≥2的组内排序|常用二元对|仅支持二元成对|
|训练复杂度|低（无Critic训练）|高（Actor + Critic双网络）|低（端到端损失）|
|正则方式|KL约束+裁剪|裁剪为主，可选价值函数正则|KL约束|

### 六、典型应用场景
1.  **LLM偏好对齐（RLHF）**：处理多候选排序标注数据，提升模型回复与人类偏好的一致性，避免reward hacking。
2.  **推理能力微调**：在数学计算、逻辑推理等可验证结果的任务中，利用二元奖励（正确/错误）做组内对比，强化模型推理稳定性，如DeepSeek - R1、DeepSeekMath的训练。
3.  **资源受限场景**：适用于无法负担Actor - Critic双网络训练的小规模模型或边缘设备微调，降低计算与内存开销。

### 七、实践要点
-  **组大小G**：常见取值8 - 16，G过小易导致归一化不稳定，G过大则增加采样与评估成本。
-  **奖励归一化**：奖励分布差异大时用标准化，差异小时仅用均值中心化；可加入clip防止极端值影响。
-  **KL正则β**：需适配任务与模型，β过小易导致模式崩溃，β过大则限制策略优化空间。
-  **参考策略选择**：优先使用高质量SFT模型作为π_ref，保障生成内容的正确性与一致性。


---

我们可以看一看AI反馈强化学习的数据集格式：

```
{
    "conversations": [
        {"role": "user", "content": "你好"},
        {"role": "assistant", "content": "空"}
    ]
}
```

为了降低该 demo 的运行门槛，在 `./demo` 文件夹下提供了包含两条 conversation 问答数据的 `rlaif_data.jsonl` 文件作为熟悉训练流程的数据集 demo。

In [12]:
# 我们可以查看一下 demo 中提供的数据
path_rlaif = './toydata/rlaif_data.jsonl'

with open(path_rlaif, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        data = json.loads(line.strip())
        print(f'Row {line_num}: {data}\n')

Row 1: {'conversations': [{'role': 'user', 'content': '列出五个基本的人格理论，并分别以一句话概括。'}, {'role': 'assistant', 'content': '空'}]}

Row 2: {'conversations': [{'role': 'user', 'content': '仔细阅读以下句子并回答“汤姆是医生还是建筑工人?”'}, {'role': 'assistant', 'content': '空'}]}



接下来，我们尝试构造 json 对象，实现对 rlaif 格式数据的读取和处理

In [13]:
class RLAIFDataset(Dataset):
    def __init__(self, jsonl_path, tokenizer, max_length=1024):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = self.load_data(jsonl_path)
        self.bos_id = tokenizer(f'{tokenizer.bos_token}assistant', add_special_tokens=False).input_ids
        self.eos_id = tokenizer(f'{tokenizer.eos_token}', add_special_tokens=False).input_ids

    def load_data(self, path):
        samples = []
        with open(path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                data = json.loads(line.strip())
                samples.append(data)
        return samples

    # 获取数据集大小
    def __len__(self):
        return len(self.samples)

    # 构建符合 ChatML 格式的对话，并返回最终的提示文本和答案
    def _create_chat_prompt(self, conversations):
        messages = []
        answer = ''
        for i, turn in enumerate(conversations):
            role = 'user' if i % 2 == 0 else 'assistant'
            messages.append({"role": role, "content": turn['content']})
            answer = turn['content']
        return self.tokenizer.apply_chat_template(
            messages[:-1],
            tokenize=False,
            add_generation_prompt=True  # 添加"assistant:"之类的生成提示符，提示模型开始回复
        ), answer

    def __getitem__(self, index):
        sample = self.samples[index]
        prompt, answer = self._create_chat_prompt(sample['conversations'])

        return {
            'prompt': prompt,
            'answer': answer
        }

In [14]:
rlaif_dataset = RLAIFDataset(path_rlaif, tokenizer)
print(f'RLAIF 数据集长度：{len(rlaif_dataset)}')
prompt, answer = rlaif_dataset[0]['prompt'], rlaif_dataset[0]['answer']
print(f'样本提示文本：\n {prompt}\n样本答案：\n{answer}')

RLAIF 数据集长度：2
样本提示文本：
 <s>system
你是 MiniMind，是一个有用的人工智能助手。</s>
<s>user
列出五个基本的人格理论，并分别以一句话概括。</s>
<s>assistant

样本答案：
空
